<a href="https://colab.research.google.com/github/Ali-J-Fesharaki/DRL_Class_Project/blob/main/SAC_LunarLanderContinuous_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y \
   libgl1-mesa-dev \
   libgl1-mesa-glx \
   libglew-dev \
   libosmesa6-dev \
   software-properties-common

!apt-get install -y patchelf

!pip install free-mujoco-py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
The following additional packages will be installed:
  libegl-dev libgl-dev libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev
  libglvnd-dev libglx-dev libopengl-dev libosmesa6
The following NEW packages will be installed:
  libegl-dev libgl-dev libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6
  libosmesa6-dev
0 upgraded, 15 newly installed, 0 to remove and 32 not upgraded.
Need to get 3,952 kB of archives.
After this operation, 18.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%mkdir -p /content/drive/MyDrive/DRL_Project/Part2/
%cd /content/drive/MyDrive/DRL_Project/Part2/

In [ ]:
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

In [ ]:
from typing import Any, ClassVar, Dict, List, Optional, Tuple, Type, TypeVar, Union

import numpy as np
import torch as th
from gymnasium import spaces
from torch.nn import functional as F

from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.noise import ActionNoise
from stable_baselines3.common.off_policy_algorithm import OffPolicyAlgorithm
from stable_baselines3.common.policies import BasePolicy, ContinuousCritic
from stable_baselines3.common.type_aliases import GymEnv, MaybeCallback, Schedule
from stable_baselines3.common.utils import get_parameters_by_name, polyak_update
from stable_baselines3.sac.policies import Actor, CnnPolicy, MlpPolicy, MultiInputPolicy, SACPolicy

SelfSAC = TypeVar("SelfSAC", bound="SAC")


class SACWithDynamicEntCoef(OffPolicyAlgorithm):
    policy_aliases: ClassVar[Dict[str, Type[BasePolicy]]] = {
        "MlpPolicy": MlpPolicy,
        "CnnPolicy": CnnPolicy,
        "MultiInputPolicy": MultiInputPolicy,
    }
    policy: SACPolicy
    actor: Actor
    critic: ContinuousCritic
    critic_target: ContinuousCritic

    def __init__(
        self,
        policy: Union[str, Type[SACPolicy]],
        env: Union[GymEnv, str],
        learning_rate: Union[float, Schedule] = 3e-4,
        buffer_size: int = 1_000_000,  # 1e6
        learning_starts: int = 100,
        batch_size: int = 256,
        tau: float = 0.005,
        gamma: float = 0.99,
        train_freq: Union[int, Tuple[int, str]] = 1,
        gradient_steps: int = 1,
        action_noise: Optional[ActionNoise] = None,
        replay_buffer_class: Optional[Type[ReplayBuffer]] = None,
        replay_buffer_kwargs: Optional[Dict[str, Any]] = None,
        optimize_memory_usage: bool = False,
        ent_coef: Union[str, float] = "auto",
        decay_ent_coef_type: str="linear",
        decay_ent_coef_rate: float=0.999,
        target_update_interval: int = 1,
        target_entropy: Union[str, float] = "auto",
        use_sde: bool = False,
        sde_sample_freq: int = -1,
        use_sde_at_warmup: bool = False,
        stats_window_size: int = 100,
        tensorboard_log: Optional[str] = None,
        policy_kwargs: Optional[Dict[str, Any]] = None,
        verbose: int = 0,
        seed: Optional[int] = None,
        device: Union[th.device, str] = "auto",
        _init_setup_model: bool = True,
    ):
        super().__init__(
            policy,
            env,
            learning_rate,
            buffer_size,
            learning_starts,
            batch_size,
            tau,
            gamma,
            train_freq,
            gradient_steps,
            action_noise,
            replay_buffer_class=replay_buffer_class,
            replay_buffer_kwargs=replay_buffer_kwargs,
            policy_kwargs=policy_kwargs,
            stats_window_size=stats_window_size,
            tensorboard_log=tensorboard_log,
            verbose=verbose,
            device=device,
            seed=seed,
            use_sde=use_sde,
            sde_sample_freq=sde_sample_freq,
            use_sde_at_warmup=use_sde_at_warmup,
            optimize_memory_usage=optimize_memory_usage,
            supported_action_spaces=(spaces.Box,),
            support_multi_env=True,
        )

        self.target_entropy = target_entropy
        self.log_ent_coef = None
        self.ent_coef = ent_coef
        self.decay_ent_coef_type = decay_ent_coef_type
        self.decay_ent_coef_rate = decay_ent_coef_rate
        self.target_update_interval = target_update_interval
        self.ent_coef_optimizer: Optional[th.optim.Adam] = None

        if _init_setup_model:
            self._setup_model()

    def _setup_model(self) -> None:
        super()._setup_model()
        self._create_aliases()
        self.batch_norm_stats = get_parameters_by_name(self.critic, ["running_"])
        self.batch_norm_stats_target = get_parameters_by_name(self.critic_target, ["running_"])
        if self.target_entropy == "auto":
            self.target_entropy = float(-np.prod(self.env.action_space.shape).astype(np.float32))
        else:
            self.target_entropy = float(self.target_entropy)

        if isinstance(self.ent_coef, str) and self.ent_coef.startswith("auto"):
            init_value = 1.0
            if "_" in self.ent_coef:
                init_value = float(self.ent_coef.split("_")[1])
                assert init_value > 0.0, "The initial value of ent_coef must be greater than 0"

            self.log_ent_coef = th.log(th.ones(1, device=self.device) * init_value).requires_grad_(True)
            self.ent_coef_optimizer = th.optim.Adam([self.log_ent_coef], lr=self.lr_schedule(1))
        else:
            self.ent_coef_tensor = th.tensor(float(self.ent_coef), device=self.device)

    def _create_aliases(self) -> None:
        self.actor = self.policy.actor
        self.critic = self.policy.critic
        self.critic_target = self.policy.critic_target

    def _compute_ent_coef(self):
        if self.decay_ent_coef_type == "linear":
            initial_value = self.ent_coef
            decrease_rate = self.decay_ent_coef_rate
            min_value = 1/ self._total_timesteps
            ent_coef = max(initial_value - decrease_rate * (self._n_updates+1), min_value)

            self.ent_coef_tensor = th.tensor(float(ent_coef), device=self.device)
        elif self.decay_ent_coef_type == "exponential":
            self.ent_coef = self.decay_ent_coef_rate*self.ent_coef
            self.ent_coef_tensor = th.tensor(float(self.ent_coef), device=self.device)

        else:
            raise NotImplementedError(f"Unknown type of ent_coef decay {self.decay_ent_coef}")

    def train(self, gradient_steps: int, batch_size: int = 64) -> None:
        self.policy.set_training_mode(True)
        optimizers = [self.actor.optimizer, self.critic.optimizer]
        if self.ent_coef_optimizer is not None:
            optimizers += [self.ent_coef_optimizer]

        self._update_learning_rate(optimizers)

        ent_coef_losses, ent_coefs = [], []
        actor_losses, critic_losses = [], []
        for gradient_step in range(gradient_steps):
            replay_data = self.replay_buffer.sample(batch_size, env=self._vec_normalize_env)

            if self.use_sde:
                self.actor.reset_noise()

            actions_pi, log_prob = self.actor.action_log_prob(replay_data.observations)
            log_prob = log_prob.reshape(-1, 1)


            self._compute_ent_coef()
            ent_coef = self.ent_coef_tensor
            ent_coefs.append(ent_coef.item())
            with th.no_grad():
                next_actions, next_log_prob = self.actor.action_log_prob(replay_data.next_observations)
                next_q_values = th.cat(self.critic_target(replay_data.next_observations, next_actions), dim=1)
                next_q_values, _ = th.min(next_q_values, dim=1, keepdim=True)
                next_q_values = next_q_values - ent_coef * next_log_prob.reshape(-1, 1)
                target_q_values = replay_data.rewards + (1 - replay_data.dones) * self.gamma * next_q_values

            current_q_values = self.critic(replay_data.observations, replay_data.actions)

            critic_loss = 0.5 * sum(F.mse_loss(current_q, target_q_values) for current_q in current_q_values)
            assert isinstance(critic_loss, th.Tensor)
            critic_losses.append(critic_loss.item())

            self.critic.optimizer.zero_grad()
            critic_loss.backward()
            self.critic.optimizer.step()

            q_values_pi = th.cat(self.critic(replay_data.observations, actions_pi), dim=1)
            min_qf_pi, _ = th.min(q_values_pi, dim=1, keepdim=True)
            actor_loss = (ent_coef * log_prob - min_qf_pi).mean()
            actor_losses.append(actor_loss.item())

            self.actor.optimizer.zero_grad()
            actor_loss.backward()
            self.actor.optimizer.step()

            if gradient_step % self.target_update_interval == 0:
                polyak_update(self.critic.parameters(), self.critic_target.parameters(), self.tau)
                polyak_update(self.batch_norm_stats, self.batch_norm_stats_target, 1.0)

        self._n_updates += gradient_steps

        self.logger.record("train/n_updates", self._n_updates, exclude="tensorboard")
        self.logger.record("train/ent_coef", np.mean(ent_coefs))
        self.logger.record("train/actor_loss", np.mean(actor_losses))
        self.logger.record("train/critic_loss", np.mean(critic_losses))
        if len(ent_coef_losses) > 0:
            self.logger.record("train/ent_coef_loss", np.mean(ent_coef_losses))

    def learn(
        self: SelfSAC,
        total_timesteps: int,
        callback: MaybeCallback = None,
        log_interval: int = 4,
        tb_log_name: str = "SAC",
        reset_num_timesteps: bool = True,
        progress_bar: bool = False,
    ) -> SelfSAC:
        return super().learn(
            total_timesteps=total_timesteps,
            callback=callback,
            log_interval=log_interval,
            tb_log_name=tb_log_name,
            reset_num_timesteps=reset_num_timesteps,
            progress_bar=progress_bar,
        )

    def _excluded_save_params(self) -> List[str]:
        return super()._excluded_save_params() + ["actor", "critic", "critic_target"]

    def _get_torch_save_params(self) -> Tuple[List[str], List[str]]:
        state_dicts = ["policy", "actor.optimizer", "critic.optimizer"]
        if self.ent_coef_optimizer is not None:
            saved_pytorch_variables = ["log_ent_coef"]
            state_dicts.append("ent_coef_optimizer")
        else:
            saved_pytorch_variables = ["ent_coef_tensor"]
        return state_dicts, saved_pytorch_variables


In [ ]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
import math
import torch
total_timesteps = 1000000
n_envs=4
base_log_dir = "./train_logs/"
dev = torch.device("cuda")
# Environments
env_names = [ "Ant-v2" ]

for env_name in env_names:
    # Create the environment
    env = make_vec_env(env_name, n_envs=n_envs)

    # Linear decay
    model_linear = SACWithDynamicEntCoef(
        "MlpPolicy",
        env,
        ent_coef=1,
        tensorboard_log=f"{base_log_dir}/{env_name}_linear",  # Include environment name
        decay_ent_coef_type="linear",
        decay_ent_coef_rate=n_envs*(1/total_timesteps)
    )
    n_training_envs = 1 # adjust eval freq depending on the number of training envs
    # Evaluate the model every 1000 steps on 5 test episodes and save the evaluation to the logs folder
    eval_callback = EvalCallback(env, eval_freq=10000 // n_training_envs, n_eval_episodes=50, log_path=(f"./logs/{env_name}_linear"))
    model_linear.learn(total_timesteps=total_timesteps, log_interval=1, progress_bar=True,callback=eval_callback)
    model_linear.save(f"SAC_dyn_ent_coef_{env_name}_linear")
    #/////////////////////////////////////////////////////////////////////////////////////////////

for env_name in env_names:
    env = make_vec_env(env_name, n_envs=n_envs)
    # exponential decay
    model_exp = SACWithDynamicEntCoef(
        "MlpPolicy",
        env,
        ent_coef=1,
        tensorboard_log=f"{base_log_dir}/{env_name}_exponential",  # Include environment name
        decay_ent_coef_type="exponential",
        decay_ent_coef_rate=math.pow((1-(1/total_timesteps)),(n_envs*math.log(total_timesteps)))
    )
    n_training_envs = 1 # adjust eval freq depending on the number of training envs
    # Evaluate the model every 1000 steps on 5 test episodes and save the evaluation to the logs folder
    eval_callback = EvalCallback(env, eval_freq=10000 // n_training_envs, n_eval_episodes=50, log_path=(f"./logs/{env_name}_exponential"))
    model_exp.learn(total_timesteps=total_timesteps, log_interval=1, progress_bar=True,callback=eval_callback)
    model_exp.save(f"sac_custom_ent_coef_{env_name}_exponential")

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 999,996/1,000,000  [ 1:31:49 < 0:00:01 , 237 it/s ]

New best mean reward!

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
import math
import torch
total_timesteps = 1000000
n_envs=4
base_log_dir = "./train_logs/"
dev = torch.device("cuda")
# Environments
env_names = [ "Ant-v2" ]

for env_name in env_names:
    # Create the environment
    env = make_vec_env(env_name, n_envs=n_envs)

    # Linear decay
    model_linear = SAC(
        "MlpPolicy",
        env,
        tensorboard_log=f"{base_log_dir}/{env_name}_auto",  # Include environment name
    )
    n_training_envs = 1 # adjust eval freq depending on the number of training envs
    # Evaluate the model every 1000 steps on 5 test episodes and save the evaluation to the logs folder
    eval_callback = EvalCallback(env, eval_freq=10000 // n_training_envs, n_eval_episodes=50, log_path=(f"./logs/{env_name}_auto"))
    model_linear.learn(total_timesteps=total_timesteps, log_interval=1, progress_bar=True,callback=eval_callback)
    model_linear.save(f"SAC_dyn_ent_coef_{env_name}_auto")

File "<__array_function__ internals>", line 180, in savez

File "/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py", line 615, in savez
    _savez(file, args, kwds, False)

File "/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py", line 712, in _savez
    zipf = zipfile_factory(file, mode="w", compression=compression)

File "/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py", line 103, in zipfile_factory
    return zipfile.ZipFile(file, *args, **kwargs)

File "/usr/lib/python3.10/zipfile.py", line 1251, in __init__
    self.fp = io.open(file, filemode)

...

File "/usr/lib/python3.10/inspect.py", line 861, in getmodule
    file = getabsfile(object, _filename)

File "/usr/lib/python3.10/inspect.py", line 845, in getabsfile
    return os.path.normcase(os.path.abspath(_filename))

File "/usr/lib/python3.10/posixpath.py", line 384, in abspath
    cwd = os.getcwd()

OSError: [Errno 107] Transport endpoint is not connected

INFO:root:
Unfortunately, your original traceback can not be constructed.



In [ ]:
%pip install tensorboard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/DRL_Project/Part2/train_logs